## Importing Libaries

In [35]:
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow import keras
from tensorflow.keras import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report

## Data Preprocessing

In [2]:
!unzip Archive.zip

Streaming output truncated to the last 5000 lines.
  inflating: Training/no_tumor/image (48).jpg  
  inflating: __MACOSX/Training/no_tumor/._image (48).jpg  
  inflating: Training/no_tumor/image(287).jpg  
  inflating: __MACOSX/Training/no_tumor/._image(287).jpg  
  inflating: Training/no_tumor/image(2).jpg  
  inflating: __MACOSX/Training/no_tumor/._image(2).jpg  
  inflating: Training/no_tumor/image.jpg  
  inflating: __MACOSX/Training/no_tumor/._image.jpg  
  inflating: Training/no_tumor/image(157).jpg  
  inflating: __MACOSX/Training/no_tumor/._image(157).jpg  
  inflating: Training/no_tumor/image(291).jpg  
  inflating: __MACOSX/Training/no_tumor/._image(291).jpg  
  inflating: Training/no_tumor/image(100).jpg  
  inflating: __MACOSX/Training/no_tumor/._image(100).jpg  
  inflating: Training/no_tumor/image(229).jpg  
  inflating: __MACOSX/Training/no_tumor/._image(229).jpg  
  inflating: Training/no_tumor/image(70).jpg  
  inflating: __MACOSX/Training/no_tumor/._image(70).jpg  
  

In [3]:
training_images = []
training_labels = []

test_images = []
test_labels = []

train_path = ('Training/')
test_path = ('Testing/')
image_size = 299

for i in os.listdir(train_path):
    for j in os.listdir(train_path + i):
        training_images.append(cv2.resize(cv2.imread(train_path + i + '/' + j), (image_size, image_size)))
        training_labels.append(i)

for i in os.listdir(test_path):
    for j in os.listdir(test_path + i):
        test_images.append(cv2.resize(cv2.imread(test_path + i + '/' + j), (image_size, image_size)))
        test_labels.append(i)

training_images = (np.array(training_images))
test_images = (np.array(test_images))

train_labels_encoded = [0 if category == 'no_tumor' else(1 if category == 'glioma_tumor' else(2 if category=='meningioma_tumor' else 3)) for category in list(training_labels)]

test_labels_encoded = [0 if category == 'no_tumor' else(1 if category == 'glioma_tumor' else(2 if category=='meningioma_tumor' else 3)) for category in list(test_labels)]


In [4]:
# augmenting the images
image_datagen = ImageDataGenerator(
    rotation_range = 45,
    zoom_range = 0.2,
    horizontal_flip = True)

image_datagen.fit(training_images)
image_datagen.fit(test_images)

## Xception Network

In [5]:
def entryFlow(inputLayer):

  #input_layer = layers.Input(shape = inputShape)
# ------------------------------------------------------------------------------------
  x = layers.Conv2D(32, kernel_size = (3, 3), strides = (2, 2))(inputLayer)
  x = layers.BatchNormalization()(x)
  x = layers.ReLU()(x)

  x = layers.Conv2D(64, kernel_size = (3, 3))(x)
  x = layers.BatchNormalization()(x)
  x = layers.ReLU()(x)
  # ------------------------------------------------------------------------------------
  residual = layers.Conv2D(128, kernel_size = (1, 1), strides = (2, 2), padding = 'same')(x)
  residual = layers.BatchNormalization()(residual)
  # ------------------------------------------------------------------------------------
  x = layers.SeparableConv2D(128, kernel_size = (3, 3), padding = 'same')(x)
  x = layers.BatchNormalization()(x)

  x = layers.ReLU()(x)
  x = layers.SeparableConv2D(128, kernel_size = (3, 3), padding = 'same')(x)
  x = layers.BatchNormalization()(x)

  x = layers.MaxPooling2D(pool_size = (3, 3), strides = (2, 2), padding = 'same')(x)

  #concat2 = layers.concatenate([residual2, maxPool2])
  x = layers.add([residual, x])

  # ------------------------------------------------------------------------------------
  residual = layers.Conv2D(256, kernel_size = (1, 1), strides = (2, 2), padding = 'same')(x)
  residual = layers.BatchNormalization()(residual)
  # ------------------------------------------------------------------------------------
  x = layers.ReLU()(x)
  x = layers.SeparableConv2D(256, kernel_size = (3, 3), padding = 'same')(x)
  x = layers.BatchNormalization()(x)

  x = layers.ReLU()(x)
  x = layers.SeparableConv2D(256, kernel_size = (3, 3), padding = 'same')(x)
  x = layers.BatchNormalization()(x)

  x = layers.MaxPooling2D(pool_size = (3, 3), strides = (2, 2), padding = 'same')(x)

  #concat3 = layers.concatenate([residual3, maxPool3])
  x = layers.add([residual, x])
  # ------------------------------------------------------------------------------------
  residual = layers.Conv2D(728, kernel_size = (1, 1), strides = (2, 2), padding = 'same')(x)
  residual = layers.BatchNormalization()(residual)
  # ------------------------------------------------------------------------------------
  x = layers.ReLU()(x)
  x = layers.SeparableConv2D(728, kernel_size = (3, 3), padding = 'same')(x)
  x = layers.BatchNormalization()(x)

  x = layers.ReLU()(x)
  x = layers.SeparableConv2D(728, kernel_size = (3, 3), padding = 'same')(x)
  x = layers.BatchNormalization()(x)

  x = layers.MaxPooling2D(pool_size = (3, 3), strides = (2, 2), padding = 'same')(x)

  #concat4 = layers.concatenate([residual4, maxPool4])
  x = layers.add([residual, x])

  return x

def middleFlow(entryFlowOutput):

  x = layers.ReLU()(entryFlowOutput)
  x = layers.SeparableConv2D(728, kernel_size = (3, 3), padding = 'same')(x)
  x = layers.BatchNormalization()(x)

  x = layers.ReLU()(x)
  x = layers.SeparableConv2D(728, kernel_size = (3, 3), padding = 'same')(x)
  x = layers.BatchNormalization()(x)

  x = layers.ReLU()(x)
  x = layers.SeparableConv2D(728, kernel_size = (3, 3), padding = 'same')(x)
  x = layers.BatchNormalization()(x)

  x = layers.add([entryFlowOutput, x])

  return x

def exitFlow(middleFlowOutput):
  residual = layers.Conv2D(1024, kernel_size = (1, 1), strides = (2, 2), padding = 'same')(middleFlowOutput)
  residual = layers.BatchNormalization()(residual)

  x = layers.ReLU()(middleFlowOutput)
  x = layers.SeparableConv2D(1024, kernel_size = (3, 3), padding = 'same')(x)
  x = layers.BatchNormalization()(x)
  x = layers.MaxPooling2D(pool_size=(3, 3), strides = (2, 2), padding = 'same')(x)

  x = layers.add([residual, x])

  x = layers.SeparableConv2D(1536, kernel_size = (3, 3), padding = 'same')(x)
  x = layers.ReLU()(x)

  x = layers.SeparableConv2D(2048, kernel_size = (3, 3), padding = 'same')(x)
  x = layers.ReLU()(x)

  x = layers.GlobalAveragePooling2D()(x)

  return x

def Xception(inputShape, numClasses):
  input_layer = layers.Input(shape = inputShape)

  entryFlowOutput = entryFlow(input_layer)

  middleFlowOutput = middleFlow(entryFlowOutput)
  n = 7
  while n > 0:
    middleFlowOutput = middleFlow(middleFlowOutput)
    n -= 1

  exitFlowOutput = exitFlow(middleFlowOutput)

  output = layers.Dense(numClasses, activation = 'softmax')(exitFlowOutput)

  model = Model(inputs = input_layer, outputs = output)

  return model

In [6]:
model = Xception((299, 299, 3), 4)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate = 1e-5),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
history = model.fit(training_images, np.array(train_labels_encoded), epochs = 15, batch_size = 32, shuffle = True)

Epoch 1/15
90/90 [==============================] - 58s 279ms/step - loss: 1.3830 - accuracy: 0.3969
Epoch 2/15
90/90 [==============================] - 22s 249ms/step - loss: 1.3539 - accuracy: 0.5523
Epoch 3/15
90/90 [==============================] - 22s 249ms/step - loss: 1.1661 - accuracy: 0.6310
Epoch 4/15
90/90 [==============================] - 22s 249ms/step - loss: 0.7795 - accuracy: 0.7230
Epoch 5/15
90/90 [==============================] - 22s 249ms/step - loss: 0.5121 - accuracy: 0.8213
Epoch 6/15
90/90 [==============================] - 22s 249ms/step - loss: 0.3703 - accuracy: 0.8707
Epoch 7/15
90/90 [==============================] - 22s 249ms/step - loss: 0.2688 - accuracy: 0.9118
Epoch 8/15
90/90 [==============================] - 22s 249ms/step - loss: 0.1912 - accuracy: 0.9383
Epoch 9/15
90/90 [==============================] - 22s 249ms/step - loss: 0.1413 - accuracy: 0.9547
Epoch 10/15
90/90 [==============================] - 22s 249ms/step - loss: 0.1178 - accura

In [37]:
def predict(model, testImages):
  predictedTarget = []
  for image in testImages:
    image = tf.expand_dims(image, axis = 0)
    predictedVal = model.predict(image, verbose = 0)
    predictedVal = tf.argmax(predictedVal, axis = -1)
    predictedTarget.append(predictedVal.numpy().tolist()[0])

  return predictedTarget

predictedTarget = predict(model, test_images)

# metrics
print(classification_report(test_labels_encoded, predictedTarget))

              precision    recall  f1-score   support

           0       0.70      1.00      0.82       105
           1       1.00      0.19      0.32       100
           2       0.74      0.96      0.83       115
           3       0.88      0.89      0.89        74

    accuracy                           0.76       394
   macro avg       0.83      0.76      0.71       394
weighted avg       0.82      0.76      0.71       394

